In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import winsound  # For Windows platform. For other platforms, alternative libraries can be used.

# Load the saved model
loaded_model = load_model("C:/Users/user/Desktop/CustomisedCNNModel (1).h5")

# Function to preprocess the image for prediction
def preprocess_image(img, target_size):
    img = cv2.resize(img, target_size)
    img = img.astype('float32') / 255.0  # Convert image to float32 and normalize
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

# Function to make predictions and classify as violent or not violent
def predict_violence_class(image, threshold=0.5):
    img = preprocess_image(image, target_size=(150, 150))
    prediction = loaded_model.predict(img)
    probability = prediction[0][0]  # Get the probability of class 1
    print(probability)
    if probability >= threshold:
        return "violent", probability
    else:
        return "notviolent", probability

# Open webcam capture
cap = cv2.VideoCapture(0)

# Define alert sound path
alert_sound_path = 'C:/Users/user/Desktop/mixkit-atm-cash-machine-key-press-2841.wav'

while True:
    ret, frame = cap.read()
    print("ret:", ret)
    if not ret:
        break

    # Predict violence class for the current frame
    class_prediction, _ = predict_violence_class(frame)

    # Draw bounding box around detected event
    if class_prediction == "violent":
        cv2.rectangle(frame, (0, 0), (frame.shape[1], frame.shape[0]), (0, 0, 255), thickness=2)  # Red color for violent event

        # Play alert sound
        winsound.PlaySound(alert_sound_path, winsound.SND_ASYNC | winsound.SND_LOOP)  # Play sound asynchronously and loop until stopped
    else:
        # Stop alert sound if not violent
        winsound.PlaySound(None, winsound.SND_ASYNC)  # Stop playing sound asynchronously

    # Overlay class name on the frame
    cv2.putText(frame, class_prediction, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display the frame
    cv2.imshow('Webcam', frame)
    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    

# Release video capture
cap.release()
cv2.destroyAllWindows()
